In [1]:
import os
import pandas as pd

## Using the variables to get data in dataframes

In [2]:
raw_data_path = os.path.abspath('../data/00_raw')
interim_data_path = os.path.abspath('../data/01_interim')

__This additional step is used to clear out the excess files from the system after, the data has been extracted and stored within a SQL Database__. However the `src/data_acquisition/01_data_acquisition.py` needs to be run first as the file download verification shall fail after this.

In [4]:
for file in os.listdir(raw_data_path):
    if file.lower().endswith('.zip'):
        file_path = os.path.join(raw_data_path, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted:{file}")

In [3]:
cdc_data = pd.read_sas(os.path.join(raw_data_path, 'DEMO_L.xpt'))
uci_diabetes_data = pd.read_csv(os.path.join(raw_data_path, 'diabetic_data.csv'))
hdhi_admissions = pd.read_csv(os.path.join(raw_data_path, 'HDHI Admission data.csv'))
hdhi_mortality = pd.read_csv(os.path.join(raw_data_path, 'HDHI Mortality Data.csv'))
hdhi_pollution = pd.read_csv(os.path.join(raw_data_path, 'HDHI Pollution Data.csv'))
mapping_data = pd.read_csv(os.path.join(raw_data_path, 'IDS_mapping.csv'))
heading_data = pd.read_csv(os.path.join(raw_data_path, 'table_headings.csv'))

In [4]:
cdc_data.head()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVSTRA,SDMVPSU,INDFMPIR
0,130378.0,12.0,2.0,1.0,43.0,NaN,5.0,6.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,50055.450807,54374.463898,173.0,2.0,5.00
1,130379.0,12.0,2.0,1.0,66.0,NaN,3.0,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,29087.450605,34084.721548,173.0,2.0,5.00
2,130380.0,12.0,2.0,2.0,44.0,NaN,2.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,80062.674301,81196.277992,174.0,1.0,1.41
3,130381.0,12.0,2.0,2.0,5.0,NaN,5.0,7.0,1.0,71.0,...,2.0,2.0,2.0,3.0,NaN,38807.268902,55698.607106,182.0,2.0,1.53
4,130382.0,12.0,2.0,1.0,2.0,NaN,3.0,3.0,2.0,34.0,...,2.0,2.0,3.0,1.0,2.0,30607.519774,36434.146346,182.0,2.0,3.60


In [5]:
uci_diabetes_data.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [6]:
hdhi_admissions.head()

,SNO,MRD No.,D.O.A,D.O.D,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,month year,DURATION OF STAY,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,1,234735,4/1/2017,4/3/2017,81,M,R,E,Apr-17,3,...,0,0,0,0,0,0,0,0,0,0
1,2,234696,4/1/2017,4/5/2017,65,M,R,E,Apr-17,5,...,0,0,0,0,0,0,0,0,0,0
2,3,234882,4/1/2017,4/3/2017,53,M,U,E,Apr-17,3,...,0,0,0,0,0,0,0,0,0,0
3,4,234635,4/1/2017,4/8/2017,67,F,U,E,Apr-17,8,...,0,0,0,0,0,0,0,0,0,0
4,5,234486,4/1/2017,4/23/2017,60,F,U,E,Apr-17,23,...,0,0,0,0,0,0,0,0,0,0


In [7]:
hdhi_pollution.head()

,DATE,AQI,PM2.5 AVG,PM2.5 MIN,PM2.5 MAX,PM10 AVG,PM10 MIN,PM10 MAX,NO2 AVG,NO2 MIN,...,CO AVG,CO MIN,CO MAX,OZONE AVG,OZONE MIN,OZONE MAX,PROMINENT POLLUTENT,MAX TEMP,MIN TEMP,HUMIDITY
0,3/25/2017,75,75,45,210,74,50,110,28,40,...,34,1,49,38,21,70,PM2.5,36,18,32.0
1,3/26/2017,82,82,47,153,75,65,86,29,26,...,32,7,55,30,8,70,PM2.5,38,19,23.0
2,3/27/2017,72,72,63,150,71,45,98,38,34,...,40,10,68,26,18,53,PM2.5,40,20,14.0
3,3/28/2017,80,78,63,88,80,73,87,39,37,...,25,12,48,21,4,71,PM10,40,21,36.0
4,3/29/2017,73,64,53,75,73,63,86,38,36,...,35,20,54,32,6,56,PM10,40,23,31.0


In [8]:
hdhi_mortality.head()

,S.NO,MRD,AGE,GENDER,RURAL/URBAN,DATE OF BROUGHT DEAD
0,353,235134,64,M,U,4/2/2017
1,301,235138,50,F,R,4/2/2017
2,302,236880,78,M,R,4/4/2017
3,278,238418,67,M,U,4/6/2017
4,276,239684,74,M,U,4/8/2017


In [9]:
mapping_data.head()

,admission_type_id,description
0,1,Emergency
1,2,Urgent
2,3,Elective
3,4,Newborn
4,5,Not Available


In [10]:
heading_data.head()

,Table Heading,Explanatory Name
0,SNO,Serial Number
1,MRD No.,Admission Number
2,D.O.A,Date of Admission
3,D.O.D,Date of Discharge
4,AGE,AGE


'd:\\Projects\\EHR-project\\data\\01_interim'